# Project: 제32회 ADP 실기시험 코드북

# 환경설정

In [1]:
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

plt.rc('font', family='Malgun Gothic') # 한글 폰트 설정
plt.rcParams['axes.unicode_minus'] = False # minus 폰트 깨짐 해결

import seaborn as sns

# 머신러닝

## EDA

In [ ]:
# 데이터 첫 3행 확인
df.head(3)

# 데이터 구조 확인
n_row, n_col = df.shape[0], df.shape[1]
print(f"데이터: {n_row}행, {n_col}열")

# 컬럼별 자료형/결측치 확인
df.info()

# 수치형 변수 기술통계
numeric_cols = df._get_numeric_data().columns.tolist()
print(f"수치형 변수: {len(numeric_cols)}개 \n {numeric_cols}")

df.describe(include=['float64', 'int64'])

# 범주형 변수 기술통계
category_cols = list(set(df.columns) - set(numeric_cols))
print(f"범주형 변수: {len(category_cols)}개 \n {category_cols}")

df.describe(include=['category'])

In [ ]:
# 결측치 확인
null_sum = df.isnull().sum()
print(null_sum)

plt.barh(null_sum.index, null_sum.values)
plt.show()

In [ ]:
# 종속변수 분포 확인
# 연속형
df['y'].hist()

# 범주형
print(df['y'].value_counts(normalize=True))
sns.countplot(df['y'])
plt.show()

In [ ]:
def make_plot(df):
    plt.figure(figsize=(20, 10))
    for i, col in enumerate(df.select_dtypes(exclude=object).columns, start=1):
        plt.subplot(3,4,i)
        plt.title(f'{col} Count')
        plt.hist(df[col])
        start_num = i+1
    for i, col in enumerate(df.select_dtypes(include=object).columns, start=start_num):
        plt.subplot(3, 4, i)
        plt.title(f'{col} Count')
        sns.countplot(data=df, x=col)
    plt.tight_layout()
    plt.show()
    
make_plot(df)

## 전처리 - 이상치 처리

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# IQR 기법을 활용한 이상치 탐색 및 조정(Winsorizing) 방식으로 이상치 처리 후 전/후 비교
def outlier_iqr(df, col, winsorizing=False):
    q1, q3 = np.quantile(df[col], 0.25), np.quantile(df[col], 0.75)
    iqr = q3 - q1
    
    upper_bound = q3 + iqr * 1.5
    lower_bound = q1 - iqr * 1.5

    # upper_bound 보다 크고 lower_bound 보다 작은 데이터의 인덱스를 찾음
    outlier_idx = df.loc[
        (df[col] >= upper_bound) |
        (df[col] <= lower_bound)
    ].index

    print(f"이상치(Outlier) 갯수: {len(outlier_idx)}개")
    print(f"이상치(Outlier) 인덱스 예시: {outlier_idx.tolist()[:10]}")
    
    if winsorizing==True:
        df_winsor = df[[col]].clip(lower_bound, upper_bound)
        
        plt.rcParams['figure.figsize'] = (10, 5)
        plt.rcParams['font.size'] = 12
        plt.rcParams['axes.unicode_minus'] = False
        
        fig, axes = plt.subplots(1, 2)
        
        for data, ax, title in zip([df[[col]], df_winsor], axes, ['Before', 'After']):
            data.boxplot(ax=ax)
            ax.set_title(f"{title}, {len(data)}")

        plt.show()
        
        df[col] = df_winsor.values
        
        return df
        
numeric_cols = df._get_numeric_data().columns

for col in numeric_cols:
    print(col)
    df = outlier_iqr(df, col, winsorizing=True)

# 기타

## 최적화 - 선형계획법

### 최소 배송비용 구하기
- (A, B, C) 3개의 공장에서 생산한 제품을 (x, y, z) 3지역으로 가능한 조건에 맞게 최대로 배송한다.
- 각 공장과 각 지역 사이의 1개 제품 배송 비용이 아래와 같을 때, 최소 배송비용은 얼마인가?

|공장|x|y|z|총 생산량|
|-|-|-|-|-|
|A|20만원|5만원|35만원|75개|
|B|15만원|2만원|20만원|55개|
|C|3만원|14만원|17만원|60개|
|지역별 수요량|80개|50개|60개|-|

In [13]:
# !pip install pulp --no-deps --user
import pulp

# 변수설정 (공장-지역) 공급량
Ax = pulp.LpVariable(name="Ax", lowBound=0)
Ay = pulp.LpVariable(name="Ay", lowBound=0)
Az = pulp.LpVariable(name="Az", lowBound=0)

Bx = pulp.LpVariable(name="Bx", lowBound=0)
By = pulp.LpVariable(name="By", lowBound=0)
Bz = pulp.LpVariable(name="Bz", lowBound=0)

Cx = pulp.LpVariable(name="Cx", lowBound=0)
Cy = pulp.LpVariable(name="Cy", lowBound=0)
Cz = pulp.LpVariable(name="Cz", lowBound=0)

# 모형적합: 제한조건을 만족하는 최적해 탐색 (최소 배송비용)
model = pulp.LpProblem(name="model", sense=pulp.LpMinimize)

# 목적함수: 최소 배송비용
model.objective = (Ax*20) + (Ay*5)  + (Az*35) + \
                  (Bx*15) + (By*2)  + (Bz*20) + \
                  (Cx*3)  + (Cy*14) + (Cz*17)

# 제약조건 설정
constraints = [
    # 지역별 수요량
    Ax + Bx + Cx == 80,
    Ay + By + Cy == 50,
    Az + Bz + Cz == 60,
    
    # 공장별 생산량
    Ax + Ay + Az <= 75,
    Bx + By + Bz <= 55,
    Cx + Cy + Cz <= 60
]

for i, const in enumerate(constraints):
    model.constraints[f"Constraints {i}"] = const
    
# 최적화
model.solve()

# 결과확인
for v in model.variables():
    print(f"{v} = {v.varValue}")

Ax = 25.0
Ay = 50.0
Az = 0.0
Bx = 0.0
By = 0.0
Bz = 55.0
Cx = 55.0
Cy = 0.0
Cz = 5.0


In [16]:
# 결론: 목적함수에 최적해 탐색의 결과를 대입하여 최소 배송비용을 정의
Ax = 25.0
Ay = 50.0
Az = 0.0
Bx = 0.0
By = 0.0
Bz = 55.0
Cx = 55.0
Cy = 0.0
Cz = 5.0

price = (Ax*20) + (Ay*5) + (Az*35) + (Bx*15) + (By*2) + (Bz*20) + (Cx*3) + (Cy*14) + (Cz*17)
print(f"최소 배송비용: {price}")

최소 배송비용: 2100.0
